In [2]:
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

In [2]:
!pip install mrjob

In [17]:
df = pd.read_csv('JawwyTV_df_new.csv')

In [24]:
list(df.iloc[50])

[50,
 '2018-03-26',
 8846,
 'Geostorm ',
 12,
 'MOVIE',
 0,
 0,
 'Action MOVIE Geostorm',
 'Action',
 0,
 1,
 'Geostorm',
 1]

In [ ]:
df.head()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   Unnamed: 0        1048575 non-null  int64 
 1   date_             1048575 non-null  object
 2   user_id_maped     1048575 non-null  int64 
 3   program_name      1048575 non-null  object
 4   duration_seconds  1048575 non-null  int64 
 5   program_class     1048575 non-null  object
 6   season            1048575 non-null  int64 
 7   episode           1048575 non-null  int64 
 8   program_desc      1048575 non-null  object
 9   program_genre     1048575 non-null  object
 10  series_title      1048575 non-null  int64 
 11  hd                1048575 non-null  int64 
 12  name              1048575 non-null  object
 13  rating            1048575 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 112.0+ MB


In [6]:
df.columns

Index(['Unnamed: 0', 'date_', 'user_id_maped', 'program_name',
       'duration_seconds', 'program_class', 'season', 'episode',
       'program_desc', 'program_genre', 'series_title', 'hd', 'name',
       'rating'],
      dtype='object')

In [14]:
%%file count_rating_each_movie.py
#Magic function that saves the code cell as a file

from mrjob.job import MRJob #import the mrjob library
from mrjob.step import MRStep #import the MRStep library
import csv #import the csv library
import time

#store columns names
columns = ['Unnamed: 0', 'date_', 'user_id_maped', 'program_name',
       'duration_seconds', 'program_class', 'season', 'episode',
       'program_desc', 'program_genre', 'series_title', 'hd', 'name',
       'rating']

class NoRatings(MRJob): # Create a class named NoRatings (number of Ratings )
    def steps(self): # Create method named steps and pass  the mapper and the reducer for MRStep 
        return[
            MRStep(mapper=self.mapping,
            reducer=self.reducing)
        ]
    
    #Create Mapper function 
    def mapping(self, _, line): #(_, line)ignore the key and take each line of the document as the value.
        reader = csv.reader([line]) #reader from csv file line by line
        for row in reader: # for loop to read rows 
            zipped=zip(columns,row) # creates a tuple  ,,, the columns is the key , row is the values
            diction = dict(zipped)
            rate = self.return_number(diction['rating'])
            movie_name = diction['name']
            genre = diction['program_genre']
            yield ((movie_name.lower(), genre.lower()), (rate, 1))  #outputing as key value pairs
    
    def return_number(self, num):
        if num == '1':
            return 1
        elif num in '2':
            return 2
        elif num in '3':
            return 3
        elif num in '4':
            return 4
        return 0

    
    def reducing(self, key, values):
        totalrate, totalviews = 0,0
        for value in values:
            totalrate += value[0]
            totalviews += value[1]
            average = round(totalrate/totalviews,2)
        yield ((key, average), totalviews)
    
if __name__ == "__main__":
    start = time.time()
    NoRatings.run()
    end = time.time()
    print(end-start)

Overwriting count_rating_each_movie.py


In [15]:
!python count_rating_each_movie.py JawwyTV_df_new.csv

[["#followfriday","thriller"],1.78]	510
[["10 days in a madhouse","thriller"],1.71]	553
[["100 treets","drama"],1.84]	1851
[["101 dalmatians","family"],1.47]	118
[["102 dalmatians","action"],1.56]	101
[["102 dalmatians","comedy"],1.55]	71
[["11.6","drama"],2.02]	379
[["12 monkeys  ","drama"],3.13]	1547
[["12 rounds : reloaded","action"],1.85]	48
[["12 rounds","action"],1.64]	11
[["127 hours","biography"],2.43]	58
[["17 again","comedy"],2.07]	99
[["2 fil-isaaf  (t)","comedy"],2.68]	496
[["2012","action"],2.53]	17
[["21 grams","drama"],1.94]	767
[["21 jump treet","action"],2.25]	4
[["21","thriller"],1.36]	25
[["22 jump treet","comedy"],2.41]	387
[["24  ","action"],3.08]	507
[["24  ","drama"],3.18]	5741
[["24 carat  (t)","drama"],3.1]	1084
[["24 hours to live","action"],1.84]	212
[["24: live another day","drama"],2.85]	973
[["30 minutes or less","action"],2.15]	551
[["300","action"],1.33]	48
[["300: rise of an empire","action"],2.22]	219
[["4 minute mile","drama"],1.29]	7
[["47 ronin","ac

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory C:\Users\MAHMOU~1\AppData\Local\Temp\count_rating_each_movie.Mahmoud Alhassan.20221130.222217.945866
Running step 1 of 1...
job output is in C:\Users\MAHMOU~1\AppData\Local\Temp\count_rating_each_movie.Mahmoud Alhassan.20221130.222217.945866\output
Streaming final output from C:\Users\MAHMOU~1\AppData\Local\Temp\count_rating_each_movie.Mahmoud Alhassan.20221130.222217.945866\output...
Removing temp directory C:\Users\MAHMOU~1\AppData\Local\Temp\count_rating_each_movie.Mahmoud Alhassan.20221130.222217.945866...


In [9]:
import re
arr = []

with open('outout.txt', 'r') as f:
    for row in f.readlines():
        arr.append(re.sub(r'[^\w\s,.\]]', '', row).replace(']', ',').replace(',,', ',').replace('\t', '').replace('\n', '').split(','))
        

In [10]:
pd.DataFrame(arr).to_csv('cleaning.txt')

In [13]:
df = pd.read_csv('cleaning.txt', names=['Name', 'Genre', 'Rating', 'Views'])
df.sample(3)

,Name,Genre,Rating,Views
1688.0,unconscious,drama,1.91,267
1668.0,triple nine,thriller,1.81,159
1075.0,premium rush,action,2.09,512
